In [ ]:
import math

import numpy as np
import open3d as o3d
import plotly.graph_objects as go
import pyransac3d as pyrsc


def ransac(data, threshold=0.05, max_iterations=1000):
    """
    RANSAC algorithm for 3D point cloud reconstruction.
    :param data: numpy array of shape (N, 3)
    :param thresh: threshold for inliers
    :param min_points: minimum number of points to fit a plane
    :param max_iterations: maximum number of iterations
    :return: numpy array of shape (1, 4) containing parameters of the best fit plane
    """

    THRESHOLD = threshold
    MAX_ITERATIONS = max_iterations
    inliers = set()

    # randomly sample 3 points
    i, j, k = np.random.randint(0, data.shape[0] - 1, 3)
    p1, p2, p3 = data[i], data[j], data[k]
    x1, y1, z1 = p1[0], p1[1], p1[2]
    x2, y2, z2 = p2[0], p2[1], p2[2]
    x3, y3, z3 = p3[0], p3[1], p3[2]

    # compute plane parameters - ax + by + cz + d = 0
    a = (y2 - y1) * (z3 - z1) - (z2 - z1) * (y3 - y1)
    b = (z2 - z1) * (x3 - x1) - (x2 - x1) * (z3 - z1)
    c = (x2 - x1) * (y3 - y1) - (y2 - y1) * (x3 - x1)
    d = -(a * x1 + b * y1 + c * z1)

    # compute distance of all points to the plane
    norm_plane_normal = math.sqrt(a * a + b * b + c * c)
    inlier_count = 0
    best_inlier_count = 0



# read and print data using open3d
data = o3d.io.read_point_cloud("record_00348.pcd", format="pcd")
print(f"Data : {repr(np.asarray(data.points))}")

# convert data to numpy array
data = np.asarray(data.points)

# Using Plotly to plot because Open3D visualizations don't work on M1 Macbook.
# Apple deprecated OpenGL since MacOS Mojave. Open3D uses OpenGL to show visualizations.
fig = go.Figure(
    data=[
        go.Scatter3d(
            x=data[:,0], y=data[:,1], z=data[:,2], 
            mode='markers',
            marker=dict(size=1, color='blue')
        )
    ],
    layout=dict(
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False)
        )
    )
)
fig.show()

# verify result using pyransac3d
plane = pyrsc.Plane()
best_eq, _ = plane.fit(data, thresh=0.05, minPoints=3, maxIteration=1000)
print(f"Parameters of best fit plane : {repr(np.array(best_eq))}")
